<a href="https://colab.research.google.com/github/anand176/JackFruit__Ripeness-Detection-System/blob/master/JackFruitRipeness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split

In [ ]:
import os
labels=[]
file_names=os.listdir('/content/drive/MyDrive/resized')
print(file_names)
for file in file_names:
    if file[0]=='r':

        labels.append(1)
    else:

        labels.append(0)


In [ ]:
import cv2
import glob
image_directory='/content/drive/MyDrive/resized/'
image_extension=['png','jpg']
files=[]
# matching_files = glob.glob(directory + '*.jpg')
print(files)
[files.extend(glob.glob(image_directory+'*.jpeg'))]
images_array=np.asarray([cv2.imread(file) for file in files])
print(type(images_array))
# print(images_array)
print(images_array.shape)

In [ ]:
X=images_array
Y=np.asarray(labels)

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=2)

In [ ]:
X_train_scaled=X_train/255
X_test_scaled=X_test/255

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [ ]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(224,224,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
# model.compile(
#     optimizer='adam',
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=['acc']

# )

In [ ]:
model.fit(X_train_scaled,Y_train,epochs=20)

In [ ]:
score,acc=model.evaluate(X_test_scaled,Y_test)
print('test loss=',score)
print('test accuracy=',acc)

In [ ]:
from google.colab.patches import cv2_imshow
import cv2

# Attempt to read the image
input_image = cv2.imread('/content/ripe_0_33.jpeg')

# Check if image reading was successful
if input_image is not None:
    # Display the image
    cv2_imshow(input_image)
else:
    print("Failed to read the image file. Please check the file path and format.")

In [ ]:
input_image_path=input('path of img to be predicted:')
input_image=cv2.imread(input_image_path)
cv2_imshow(input_image)
input_image_resize=cv2.resize(input_image,(224,224))
print(input_image_resize)
input_image_scaled=input_image_resize/255
image_reshaped=np.reshape(input_image_scaled,[1,224,224,3])
input_prediction=model.predict(image_reshaped)
# input_pred_label=np.argmax(input_prediction)
# print(input_pred_label)
print(input_prediction[0][0])
if input_prediction[0][0]>.5:
  # .59 is just the approx value
  print('ripe')
else:
  print('unripe')